In [1]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite

print(sklearn.__version__)

0.19.0


# Let's use CoNLL 2002 data to build a NER system

CoNLL2002 corpus is available in NLTK. We use Spanish data.

In [2]:
nltk.corpus.conll2002.fileids()

['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [6]:
%%time
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

Wall time: 2.98 s


Data format:

In [4]:
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

## Features

Next, define some features. In this example we use word identity, word suffix, word shape and word POS tag; also, some information from nearby words is used. 

This makes a simple baseline, but you certainly can add and remove some features to get (much?) better results - experiment with it.

In [7]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]    

This is what word2features extracts:

In [8]:
sent2features(train_sents[0])

[['bias',
  'word.lower=melbourne',
  'word[-3:]=rne',
  'word[-2:]=ne',
  'word.isupper=False',
  'word.istitle=True',
  'word.isdigit=False',
  'postag=NP',
  'postag[:2]=NP',
  'BOS',
  '+1:word.lower=(',
  '+1:word.istitle=False',
  '+1:word.isupper=False',
  '+1:postag=Fpa',
  '+1:postag[:2]=Fp'],
 ['bias',
  'word.lower=(',
  'word[-3:]=(',
  'word[-2:]=(',
  'word.isupper=False',
  'word.istitle=False',
  'word.isdigit=False',
  'postag=Fpa',
  'postag[:2]=Fp',
  '-1:word.lower=melbourne',
  '-1:word.istitle=True',
  '-1:word.isupper=False',
  '-1:postag=NP',
  '-1:postag[:2]=NP',
  '+1:word.lower=australia',
  '+1:word.istitle=True',
  '+1:word.isupper=False',
  '+1:postag=NP',
  '+1:postag[:2]=NP'],
 ['bias',
  'word.lower=australia',
  'word[-3:]=lia',
  'word[-2:]=ia',
  'word.isupper=False',
  'word.istitle=True',
  'word.isdigit=False',
  'postag=NP',
  'postag[:2]=NP',
  '-1:word.lower=(',
  '-1:word.istitle=False',
  '-1:word.isupper=False',
  '-1:postag=Fpa',
  '-1:post

Extract the features from the data:

In [11]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

train_size=100
test_size=20
X_train=X_train[:train_size]
y_train=y_train[:train_size]
X_test=X_test[:test_size]
y_test=y_test[:test_size]


Wall time: 3.4 s


## Train the model

To train the model, we create pycrfsuite.Trainer, load the training data and call 'train' method. 
First, create pycrfsuite.Trainer and load the training data to CRFsuite:

In [14]:
%%time
trainer = pycrfsuite.Trainer(verbose=False)#verbose = debug information？
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

Wall time: 44.5 ms


Set training parameters. We will use L-BFGS training algorithm (it is default) with Elastic Net (L1 + L2) regularization.

In [16]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier
    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

Possible parameters for the default training algorithm:

In [21]:
print(trainer.params())
print(trainer.get_params())

['feature.minfreq', 'feature.possible_states', 'feature.possible_transitions', 'c1', 'c2', 'max_iterations', 'num_memories', 'epsilon', 'period', 'delta', 'linesearch', 'max_linesearch']
{'feature.minfreq': 0.0, 'feature.possible_states': False, 'feature.possible_transitions': True, 'c1': 1.0, 'c2': 0.001, 'max_iterations': 50, 'num_memories': 6, 'epsilon': 1e-05, 'period': 10, 'delta': 1e-05, 'linesearch': 'MoreThuente', 'max_linesearch': 20}


Train the model:

In [23]:
%%time
trainer.train('conll2002-esp.model')

Wall time: 183 ms


trainer.train saves model to a file:

In [12]:
!ls -lh ./conll2002-esp.crfsuite

-rw-r--r--  1 gsh25  staff   600K Jun 22 14:56 ./conll2002-esp.crfsuite


We can also get information about the final state of the model by looking at the trainer's logparser. If we had tagged our input data using the optional group argument in add, and had used the optional holdout argument during train, there would be information about the trainer's performance on the holdout set as well. 

In [24]:
trainer.logparser.last_iteration

{'active_features': 396,
 'error_norm': 9.165816,
 'feature_norm': 17.263419,
 'linesearch_step': 1.0,
 'linesearch_trials': 1,
 'loss': 321.966377,
 'num': 50,
 'scores': {},
 'time': 0.003}

We can also get this information for every step using trainer.logparser.iterations

In [26]:
print(len(trainer.logparser.iterations), trainer.logparser.iterations[-1])

50 {'num': 50, 'scores': {}, 'loss': 321.966377, 'feature_norm': 17.263419, 'error_norm': 9.165816, 'active_features': 396, 'linesearch_trials': 1, 'linesearch_step': 1.0, 'time': 0.003}


## Make predictions

To use the trained model, create pycrfsuite.Tagger, open the model and use "tag" method:

In [27]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002-esp.model')

Let's tag a sentence to see how it works:

In [31]:
example_sent = test_sents[3]
print(' '.join(sent2tokens(example_sent)), end='\n\n')

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

García Aranda presentó a la prensa el sistema Amadeus , que utilizan la mayor parte de las agencias de viajes españolas para reservar billetes de avión o tren , así como plazas de hotel , y que ahora pueden utilizar también los usuarios finales a través de Internet .

Predicted: B-ORG I-ORG O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O B-LOC O
Correct:   B-PER I-PER O O O O O O B-MISC O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O B-MISC O


## Evaluate the model

In [32]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

Predict entity labels for all sentences in our testing set ('testb' Spanish data):

In [33]:
%%time
y_pred = [tagger.tag(xseq) for xseq in X_test]

Wall time: 12 ms


..and check the result. Note this report is not comparable to results in CONLL2002 papers because here we check per-token results (not per-entity). Per-entity numbers will be worse.  

In [34]:
print(bio_classification_report(y_test, y_pred))

             precision    recall  f1-score   support

      B-LOC       0.71      0.59      0.65        17
      I-LOC       0.00      0.00      0.00         9
     B-MISC       0.29      0.25      0.27         8
     I-MISC       0.17      0.57      0.27         7
      B-ORG       0.82      0.58      0.68        24
      I-ORG       0.63      0.48      0.55        25
      B-PER       0.57      0.67      0.62         6
      I-PER       0.55      0.86      0.67         7

avg / total       0.57      0.50      0.52       103



E:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Let's check what classifier learned

In [41]:
from collections import Counter
info = tagger.info()
# print(info)
# for d in info.transitions:
#     print(d)
# print(info.state_features)
# print(info.labels)

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

Top likely transitions:
B-PER  -> I-PER   5.204776
B-ORG  -> I-ORG   4.456426
I-PER  -> I-PER   4.248969
I-LOC  -> I-LOC   4.202364
I-MISC -> I-MISC  4.188580
I-ORG  -> I-ORG   3.905730
B-MISC -> I-MISC  3.735016
B-LOC  -> I-LOC   3.355209
O      -> B-MISC  1.264860
O      -> O       1.176777
O      -> B-ORG   1.130441
O      -> B-LOC   0.161906
I-PER  -> O       0.009665
B-PER  -> O       -0.191249
I-MISC -> O       -0.325281

Top unlikely transitions:
I-MISC -> I-MISC  4.188580
I-ORG  -> I-ORG   3.905730
B-MISC -> I-MISC  3.735016
B-LOC  -> I-LOC   3.355209
O      -> B-MISC  1.264860
O      -> O       1.176777
O      -> B-ORG   1.130441
O      -> B-LOC   0.161906
I-PER  -> O       0.009665
B-PER  -> O       -0.191249
I-MISC -> O       -0.325281
O      -> I-LOC   -1.880184
O      -> I-PER   -2.084558
O      -> I-ORG   -3.117256
O      -> I-MISC  -3.271135


We can see that, for example, it is very likely that the beginning of an organization name (B-ORG) will be followed by a token inside organization name (I-ORG), but transitions to I-ORG from tokens with other labels are penalized. Also note I-PER -> B-LOC transition: a positive weight means that model thinks that a person name is often followed by a location.

Check the state features:

In [19]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

Top positive:
8.886516 B-ORG  word.lower=efe-cantabria
8.743642 B-ORG  word.lower=psoe-progresistas
5.769032 B-LOC  -1:word.lower=cantabria
5.195429 I-LOC  -1:word.lower=calle
5.116821 O      word.lower=mayo
4.990871 O      -1:word.lower=día
4.910915 I-ORG  -1:word.lower=l
4.721572 B-MISC word.lower=diversia
4.676259 B-ORG  word.lower=telefónica
4.334354 B-ORG  word[-2:]=-e
4.149862 B-ORG  word.lower=amena
4.141370 B-ORG  word.lower=terra
3.942852 O      word.istitle=False
3.926397 B-ORG  word.lower=continente
3.924672 B-ORG  word.lower=acesa
3.888706 O      word.lower=euro
3.856445 B-PER  -1:word.lower=según
3.812373 B-MISC word.lower=exteriores
3.807582 I-MISC -1:word.lower=1.9
3.807098 B-MISC word.lower=sanidad

Top negative:
-1.965379 O      word.lower=fundación
-1.981541 O      -1:word.lower=británica
-2.118347 O      word.lower=061
-2.190653 B-PER  word[-3:]=nes
-2.226373 B-ORG  postag=SP
-2.226373 B-ORG  postag[:2]=SP
-2.260972 O      word[-3:]=uia
-2.384920 O      -1:word.lower

Some observations:

* **8.743642 B-ORG  word.lower=psoe-progresistas** - the model remembered names of some entities - maybe it is overfit, or maybe our features are not adequate, or maybe remembering is indeed helpful;
* **5.195429 I-LOC  -1:word.lower=calle**: "calle" is a street in Spanish; model learns that if a previous word was "calle" then the token is likely a part of location;
* **-3.529449 O      word.isupper=True**, ** -2.913103 O      word.istitle=True **: UPPERCASED or TitleCased words are likely entities of some kind;
* **-2.585756 O      postag=NP** - proper nouns (NP is a proper noun in the Spanish tagset) are often entities.

## What to do next

1. Load 'testa' Spanish data.
2. Use it to develop better features and to find best model parameters.
3. Apply the model to 'testb' data again.

The model in this notebook is just a starting point; you certainly can do better!